# Meshing different geometries

In [ ]:
import sys
sys.path.append("..")

import numpy as np
import lapy
from lapy import TetMesh, Solver, TriaMesh
import matplotlib.pyplot as plt
import numpy as np
import skimage.measure
import pygalmesh
import meshio
from scipy.ndimage import zoom
import os

See https://github.com/meshpro/pygalmesh for examples and documentation on 3D mesh rendering.

# Geometry #1: Ellipsoid

#### Step 1: Rendering mesh

In [ ]:
class Ellipsoid(pygalmesh.DomainBase):
    def __init__(self, a, b, c):
        super().__init__()
        self.a = a
        self.b = b
        self.c = c

    def eval(self, x):
        # Equation of the ellipsoid: (x/a)^2 + (y/b)^2 + (z/c)^2 - 1
        return (x[0] / self.a) ** 2 + (x[1] / self.b) ** 2 + (x[2] / self.c) ** 2 - 1

    def get_bounding_sphere_squared_radius(self):
        # The bounding sphere radius is the largest axis length.
        return max(self.a, self.b, self.c) ** 2

In [ ]:
function = Ellipsoid(0.5, 0.5, 1)
mesh = pygalmesh.generate_mesh(function, max_cell_circumradius=0.0295)

mesh.write("../Files/ellipse.vtk")

#### Step 2: Loading mesh and generating tetrahedra

In [ ]:
mesh = meshio.read('../Files/ellipse.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

#### Step 3: Solving eigenmodes

Solving 100 eigenmodes.

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=101)

# Squeezing vertex coordinates in arbitrary [0, 1] interval (major axis length of volume = 1)
vertices -= np.min(vertices) 
vertices /= np.max(vertices)

In [ ]:
print(eigenvectors.shape)

Visualizing eigenmodes.

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(10, 10), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c=eigenvectors[:, i+1], alpha=0.5, cmap='coolwarm')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_axis_off()

plt.tight_layout(pad=0)
#plt.savefig('Figures/eigenhearts.png')
plt.show()

In [ ]:
np.save('../Files/vertices_ellipse.npy', vertices)
np.save('../Files/eigenmodes_ellipse.npy', eigenvectors)

# Geometry #2: Heart

#### Step 1: Rendering mesh

This one is based on a mathematical function.

In [ ]:
class Heart(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()

    def eval(self, x):
        return (
            (x[0] ** 2 + 9.0 / 4.0 * x[1] ** 2 + x[2] ** 2 - 1) ** 3
            - x[0] ** 2 * x[2] ** 3
            - 9.0 / 80.0 * x[1] ** 2 * x[2] ** 3
        )

    def get_bounding_sphere_squared_radius(self):
        return 10.0

In [ ]:
function = Heart()
mesh = pygalmesh.generate_mesh(function, max_cell_circumradius=0.0435)

In [ ]:
mesh.write("../Files/heart.vtk")

#### Step 2: Loading mesh and generating tetrahedra

In [ ]:
mesh = meshio.read('../Files/heart.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

#### Step 3: Solving eigenmodes

Solving 100 eigenmodes.

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=101)

# Squeezing vertex coordinates in arbitrary [0, 1] interval (major axis length of volume = 1)
vertices -= np.min(vertices) 
vertices /= np.max(vertices)

Visualizing eigenmodes.

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(10, 10), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c=eigenvectors[:, i+1], alpha=0.5, cmap='coolwarm')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_axis_off()

plt.tight_layout(pad=0)
#plt.savefig('Figures/eigenhearts.png')
plt.show()

Saving eigenmodes.

In [ ]:
np.save('../Files/vertices_heart.npy', vertices)
np.save('../Files/eigenmodes_heart.npy', eigenvectors)

# Geometry #3: Torus

In [ ]:
class Torus(pygalmesh.DomainBase):
    
    def __init__(self, R, r):
        super().__init__()
        self.R = R  # Major radius (distance from the center of the hole to the center of the tube)
        self.r = r  # Minor radius (radius of the tube)

    def eval(self, x):
        # Torus equation: ((sqrt(x^2 + y^2) - R)^2 + z^2 - r^2)
        # Using (x[0] for x, x[1] for y, x[2] for z):
        sq_dist_xy = x[0] ** 2 + x[1] ** 2
        return ((sq_dist_xy - self.R ** 2) ** 2 + 4 * self.R ** 2 * x[2] ** 2 - 4 * self.R ** 2 * self.r ** 2)

    def get_bounding_sphere_squared_radius(self):
        # The bounding sphere radius is the major radius plus the minor radius
        return (self.R + self.r) ** 2

In [ ]:
torus = Torus(R=3, r=1)
mesh = pygalmesh.generate_mesh(torus, max_cell_circumradius=0.115)

mesh.write("../Files/torus.vtk")

#### Step 2: Loading mesh and generating tetrahedra

In [ ]:
mesh = meshio.read('../Files/torus.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

#### Step 3: Solving eigenmodes

Solving 100 eigenmodes.

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=101)

# Squeezing vertex coordinates in arbitrary [0, 1] interval (major axis length of volume = 1)
vertices -= np.min(vertices) 
vertices /= np.max(vertices)

In [ ]:
print(eigenvectors.shape)

Visualizing eigenmodes.

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(10, 10), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c=eigenvectors[:, i+1], alpha=0.5, cmap='coolwarm')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_axis_off()

plt.tight_layout(pad=0)
#plt.savefig('Figures/eigenhearts.png')
plt.show()

In [ ]:
np.save('../Files/vertices_torus.npy', vertices)
np.save('../Files/eigenmodes_torus.npy', eigenvectors)

# Geometry #4: Cow

In [ ]:
mesh = pygalmesh.generate_volume_mesh_from_surface_mesh(
    "../Files/cow.obj",
    max_cell_circumradius=0.08,
    reorient=True
)

In [ ]:
#mesh = meshio.read('Files/cow.obj')

vertices = mesh.points
cells = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, cells)
print(vertices.shape)

vertices -= np.min(vertices) 
vertices /= np.max(vertices)

#### Step 2: Solving eigenmodes

Solving 100 eigenmodes.

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=101)

In [ ]:
print(eigenvectors.shape)

In [ ]:
vertices = np.stack([vertices[:, 2], vertices[:, 0], vertices[:, 1]], axis=1)

In [ ]:
fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(10, 10), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[::2, 0], vertices[::2, 1], vertices[::2, 2], c=eigenvectors[::2, i+1], alpha=0.5, cmap='coolwarm')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    #ax.set_ylabel('Y')
    #ax.set_xlabel('X')
    ax.set_axis_off()
    ax.view_init(elev=-10, azim=135)

plt.tight_layout(pad=0)
#plt.savefig('Figures/eigenhearts.png')
plt.show()

In [ ]:
np.save('../Files/vertices_cow.npy', vertices)
np.save('../Files/eigenmodes_cow.npy', eigenvectors)

# Alternative approach to render the ellipsoid

#### Step 1: Rendering mesh

Rendering a binary sphere in a (100, 100, 100)-sized array, then downscaling axes by different factors to get an ellipse. Suffers a bit from the discretization inherent to the 3D stack before converting the volume into continuous mesh coordinates.

In [ ]:
L = 100
r = 0.75

x_ = np.linspace(-1.0, 1.0, L)
y_ = np.linspace(-1.0, 1.0, L)
z_ = np.linspace(-1.0, 1.0, L)
x, y, z = np.meshgrid(x_, y_, z_)

vol = np.empty((L, L, L), dtype=np.uint8)
idx = x ** 2 + y ** 2 + z ** 2 < r ** 2
vol[idx] = 1
vol[~idx] = 0

In [ ]:
vol = zoom(vol, (1/2, 1/2, 2/2), order=1) # Downsampling by different factors in each axis

Generating mesh.

In [ ]:
voxel_size = (0.1, 0.1, 0.1)

mesh = pygalmesh.generate_from_array(
    vol, voxel_size, max_facet_distance=0.25, max_cell_circumradius=0.125
)

mesh.write("../Files/ellipse.vtk")

#### Step 2: Loading mesh and generating tetrahedra

In [ ]:
mesh = meshio.read('../Files/ellipse.vtk')

vertices = mesh.points
tetrahedra = mesh.cells_dict['tetra']

tet_mesh = TetMesh(vertices, tetrahedra)
print(vertices.shape)

#### Step 3: Solving eigenmodes

In [ ]:
solver = Solver(tet_mesh)
eigenvalues, eigenvectors = solver.eigs(k=101)

# Squeezing vertex coordinates in arbitrary [0, 1] interval (major axis length of volume = 1)
vertices -= np.min(vertices) 
vertices /= np.max(vertices)

Displaying eigenmodes.

In [ ]:
cut_volume = False
if cut_volume:
    cut = (vertices[:, 0] < 0.25) | (vertices[:, 2] < 0.5)
else:
    cut = vertices[:, 0] > -1   # Always true

fig, axes = plt.subplots(4, 5, subplot_kw={"projection": "3d"}, figsize=(10, 10), dpi=150)

for i in range(20):
    ax = axes.flatten()[i]
    ax.scatter(vertices[cut, 0] + 0.25, vertices[cut, 1] + 0.25, vertices[cut, 2], c=eigenvectors[cut, i + 1], alpha=0.5, cmap='coolwarm')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_axis_off()
    # ax.set_frame_on(False) # Older version of Matplotlib

plt.tight_layout(pad=0)
#plt.savefig('eigenhearts.png')
plt.show()

In [ ]:
np.save('../Files/vertices_ellipse.npy', vertices)
np.save('../Files/eigenmodes_ellipse.npy', eigenvectors)